## Создание собственного скрейпера

Библиотеки, которые нам понадобятся:

In [43]:
# стандартные
import re
import urllib.request
from time import sleep
# нужно установить: pip install bs4
from bs4 import BeautifulSoup as bs

Посмотрим, как выглядит заглавная страница ресурса:

In [ ]:
fid = urllib.request.urlopen('https://proza.ru/')  # откроем ее с помощью библиотеки urllib

webpage = fid.read().decode('cp1251')  # как узнать, какая нам нужна кодировка: нужно открыть в браузере эту страницу, нажать f12 (для chrome) и в терминале (console) набрать document.characterSet

print(webpage)  # посмотрим, как оно выглядит

![Image](console.PNG)

In [3]:
soup = bs(webpage, 'html.parser')  # передадим нашу страницу в парсер bs4

In [10]:
for link in soup.find_all('a'):
    # посмотрим, какие у нас есть ссылки
    print(link.get('href'))  

/
/authors/
/poems/
/board/
/search.html
https://shop.proza.ru/
https://o.proza.ru/
/login/
https://o.proza.ru/
/poems/
/editor/2021/06/25/869
/avtor/shevnin
/editor/2018/08/04/512
/avtor/chornogor
/editor/2021/06/28/1022
/avtor/molgast
/editor/2021/06/25/931
/avtor/mudamailru
/editor/2021/06/25/954
/avtor/redin1970
/editor/2021/02/02/638
/avtor/ifynfkm1971
/editor/2021/11/10/995
/avtor/peresvetm
/editor/2021/06/26/893
/avtor/olgaatp
/editor/2021/06/26/969
/avtor/medvegonok
/editor/2021/06/26/1176
/avtor/helga35
/editor/2021/12/23/1360
/avtor/franziri
/editor/2020/07/25/221
/avtor/miliku
/editor/2020/07/15/280
/avtor/iolnta
/editor/2020/08/23/761
/avtor/vlad43
/editor/2018/03/01/1782
/avtor/daryona67
/editor/2021/06/26/1341
/avtor/tchsemsha
/editor/2021/06/27/212
/avtor/vedrov8
/editor/2021/06/27/351
/avtor/sckolot
/editor/2021/06/27/961
/avtor/klassik1
/editor/2021/06/27/1564
/avtor/gorizont
/editor/2020/01/17/1277
/avtor/marivanna
/editor/
/authors/
/authors/online.html
/avtor/yurche

Нас интересуют только внутренние, а они, очевидно, не начинаются с https. Отсеем их:

In [11]:
links = []
for link in soup.find_all('a', attrs={'href': re.compile(r'^/.+')}):
    links.append('https://proza.ru' + link.get('href'))

In [12]:
links[:10] # посмотрим, что вышло

['https://proza.ru/authors/',
 'https://proza.ru/poems/',
 'https://proza.ru/board/',
 'https://proza.ru/search.html',
 'https://proza.ru/login/',
 'https://proza.ru/poems/',
 'https://proza.ru/editor/2021/06/25/869',
 'https://proza.ru/avtor/shevnin',
 'https://proza.ru/editor/2018/08/04/512',
 'https://proza.ru/avtor/chornogor']

Вроде похоже на правду. Давайте собирать уже просмотренные ссылки в множество, чтобы не попадать в них по нескольку раз:

In [14]:
viewedlinks = set('https://proza.ru/') # главную мы типа уже посмотрели

Давайте попробуем в тестовом режиме пройтись по ссылкам до упора один раз. Давайте оформим наше извлечение страницы в какую-нибудь функцию:

In [17]:
def looklinks(page, viewed):
    """Тестовая функция для просмотра всех ссылок на странице"""
    fid = urllib.request.urlopen(page) 
    webpage = fid.read().decode('cp1251')
    viewed.add(page)
    soup = bs(webpage, 'html.parser') 
    links = []
    for link in soup.find_all('a', attrs={'href': re.compile(r'^/.+')}):
        l = 'https://proza.ru' + link.get('href')
        if l not in viewed:
            links.append(l)
    return links

In [27]:
step = looklinks(links[0], viewedlinks)

In [28]:
step

['https://proza.ru/avtor/yurcher',
 'https://proza.ru/avtor/krotovalv',
 'https://proza.ru/avtor/20055',
 'https://proza.ru/avtor/iolnta',
 'https://proza.ru/avtor/tlev66',
 'https://proza.ru/avtor/lotos4',
 'https://proza.ru/avtor/akochevnik',
 'https://proza.ru/avtor/ramuleya',
 'https://proza.ru/avtor/evgeny1951mai',
 'https://proza.ru/avtor/christoforoff',
 'https://proza.ru/avtor/gurkovskiy107',
 'https://proza.ru/avtor/lynofob',
 'https://proza.ru/login/promo.html?invite']

In [26]:
step = looklinks(step[0], viewedlinks)
step

[]

Если прощелкать несколько раз, рано или поздно дойдем до пустого списка: значит, ссылки на странице закончились. Давайте для проверки, что находится на страницах с авторами, прощелкаем более избирательно:

In [33]:
# 'https://proza.ru/avtor/akochevnik'
# 'https://proza.ru/avtor/akochevnik&book=9#9'
# 'https://proza.ru/2012/11/30/1362'
# 'https://proza.ru/complain.html?text_2012/11/30/1362'
# 'https://proza.ru/about/pravila.html'

step = looklinks('https://proza.ru/about/pravila.html', viewedlinks)
step

['https://proza.ru/about/pravo/gk70.html#1274',
 'https://proza.ru/about/pravo/',
 'https://proza.ru/about/feedback.html?app']

Ну вроде как понятно: все должно работать. Нам теперь осталось понять, на страницах какого вида лежит хороший текст. Предположу, что на страницах с датами. Проверим, напринтив такую страницу:

In [36]:
fid = urllib.request.urlopen('https://proza.ru/2012/11/30/1362') 
webpage = fid.read().decode('cp1251')
print(webpage)

<head>
<meta http-equiv="Content-Type" content="text/html" />
<meta name="viewport" content="width=device-width, initial-scale=1.0" />
<meta http-equiv="X-Frame-Options" content="deny" />
<meta name="DESCRIPTION" content="Сервер предоставляет авторам возможность свободной публикации и обсуждения произведений современной прозы." />
<meta name="Keywords" content="проза произведение роман новелла миниатюра автор литература творчество лирика журнал конкурс" />
<meta name="title" content="Моё деревенское счастье. Предисловие (Александр Алексеевич Кочевник) / Проза.ру" />
<title>Моё деревенское счастье. Предисловие (Александр Алексеевич Кочевник) / Проза.ру</title>
<link rel="stylesheet" href="/styles/s_main_11.css">
<script language="JavaScript" src="/styles/p_main_1.js"></script>
</head>

<body bgcolor="#FFFFFF" text="#000000" link="#000080" vlink="#505050" alink="#FF0000" marginheight="0" marginwidth="0" topmargin="0" leftmargin="0">



<div id="container">

  <div id="header">
    <div c

Действительно: тут лежат наши тексты. Возможно, они еще где-нибудь лежат (poem?), это тоже можно проверить. Ну предположим, что мы все проверили и убедились, что страницы с текстами произведений - это, очевидно, адреса в формате r'\d{4}/\d\d/\d\d/.*' = '1234/56/78/123...'

Давайте попробуем допилить нашу тестовую функцию таким образом, чтобы она дополнительно проверяла, не является ли просматриваемая страница текстовой, и добавляла ее в отдельный список. 

In [46]:
texts = []

In [47]:
def looklinks(page, viewed, texts):
    """Тестовая функция для просмотра всех ссылок на странице и добавления текстов"""
    print(page)
    fid = urllib.request.urlopen(page) 
    webpage = fid.read().decode('cp1251')
    viewed.add(page)
    if re.match(r'https://proza.ru/\d{4}/\d\d/\d\d/.*', page):
        texts.append(webpage)
        print(f'text for {page} added!')
    soup = bs(webpage, 'html.parser') 
    links = []
    for link in soup.find_all('a', attrs={'href': re.compile(r'^/.+')}):
        l = 'https://proza.ru' + link.get('href')
        if l not in viewed:
            links.append(l)
    if links:
        for link in links:
            sleep(1)  # чтоб нас случайно не заблочили как робота
            looklinks(link, viewed, texts)
    else:
        return 

Потестим нашу функцию для какого-нибудь конкретного автора:

In [48]:
start = 'https://proza.ru/avtor/evgeny1951mai'
looklinks(start, viewedlinks, texts)


https://proza.ru/avtor/evgeny1951mai
https://proza.ru/rec_writer.html?evgeny1951mai
https://proza.ru/board/list.html?rec_writer=evgeny1951mai&list=short
https://proza.ru/board/list.html?rec_writer=evgeny1951mai
https://proza.ru/2020/10/03/928
text for https://proza.ru/2020/10/03/928 added!
https://proza.ru/avtor/victorlev
https://proza.ru/login/messages.html?victorlev
https://proza.ru/recommend.html?victorlev
https://proza.ru/rec_author.html?victorlev
https://proza.ru/board/list.html?rec_author=victorlev&list=short
https://proza.ru/board/list.html?rec_author=victorlev
https://proza.ru/rec.html?2022/10/09/657
https://proza.ru/complain.html?rec_2022/10/09/657
https://proza.ru/login/profile.html?blacklist
https://proza.ru/addnotes.html?2022/10/09/657
https://proza.ru/cgi-bin/reg.pl
https://proza.ru/about/dogovor.html
https://proza.ru/about/pravo/
https://proza.ru/about/pravo/gk70.html#1265
https://proza.ru/about/pravo/
https://proza.ru/about/pravo/
https://proza.ru/about/feedback.html
htt

KeyboardInterrupt: 

In [ ]:
texts

Видим, что все работает и что наш скрейпер, как ему и полагается, ползает со страницы на страницу и рано или поздно выкачает всю прозу, если мы не установим какой-нибудь лимит, например:

In [ ]:
def looklinks(page, viewed, texts, N=10):
    """Тестовая функция для просмотра всех ссылок на странице и добавления текстов"""
    if len(texts) > N:  # N - число текстов, которое мы хотим выкачать
        return
    print(page)
    fid = urllib.request.urlopen(page) 
    webpage = fid.read().decode('cp1251')
    viewed.add(page)
    if re.match(r'https://proza.ru/\d{4}/\d\d/\d\d/.*', page):
        texts.append(webpage)
        print(f'text for {page} added!')
    soup = bs(webpage, 'html.parser') 
    links = []
    for link in soup.find_all('a', attrs={'href': re.compile(r'^/.+')}):
        l = 'https://proza.ru' + link.get('href')
        if l not in viewed:
            links.append(l)
    if links:
        for link in links:
            sleep(1)  # чтоб нас случайно не заблочили как робота
            looklinks(link, viewed, texts)
    else:
        return 

Отлично, дело за малым: вытащить собственно тексты из html-разметки :) И метаинформацию о них: заголовок, автора и дату публикации. 

Заголовок, очевидно, лежит в самом html, это:

        <title>Моё деревенское счастье. Предисловие (Александр Алексеевич Кочевник) / Проза.ру</title> 

Автор тоже есть, он зашит в заголовке: r'(.+?) \((.+?)\) / Проза.ру'

Ну а дату можно из ссылки страницы извлечь, мы уже выяснили, что эти ссылки нумеруются датами. Следовательно:

In [78]:
def looklinks(page, viewed, texts, N=10):
    """Тестовая функция для просмотра всех ссылок на странице и добавления текстов"""
    print(page)
    fid = urllib.request.urlopen(page) 
    webpage = fid.read().decode('cp1251')
    viewed.add(page)
    soup = bs(webpage, 'html.parser') 
    # вот наш кусочек, который будет отвечать за добавление всего в итоговый список текстов:
    if re.match(r'https://proza.ru/\d{4}/\d\d/\d\d/.*', page):
        ta = re.search(r'(.+?) \((.+?)\) / Проза.ру', soup.title.string) # расчленяем заголовок на автора и название с помощью групп в регулярках
        if ta:
            title = ta.group(1)
            author = ta.group(2)
        else:
            # а вдруг мы где-то накосячили с регуляркой
            title = 'Unknown'
            author = 'Unknown'
        year = re.search(r'https://proza.ru/(\d{4}/\d\d/\d\d)/.*', page).group(1) # получаем год с помощью группы
        text = '\n'.join((elem.text for elem in soup.find_all("div", {"class": "text"})))  # собираем все возможные сегменты класса "текст" с помощью bs4
        texts.append({'title': title, 'author': author, 'year': year, 'text': text})
        print(f'text for {page} added!')
    links = []
    for link in soup.find_all('a', attrs={'href': re.compile(r'^/.+')}):
        l = 'https://proza.ru' + link.get('href')
        if l not in viewed:
            links.append(l)
    if links:
        for link in links:
            if re.search('board|login|help|topic|type|about', link): # мы, вероятно, не хотим бесконечно лазать по внутренностям не интересующих нас разделов
                continue
            if len(texts) > N:  # N - число текстов, которое мы хотим выкачать
                return
            sleep(0.5)  # чтоб нас случайно не заблочили как робота
            looklinks(link, viewed, texts, N)
    else:
        return 

In [79]:
viewedlinks = set('https://proza.ru/') # для чистоты эксперимента обнулим наш список просмотренного
texts = [] # и список текстов

In [80]:
start = 'https://proza.ru/avtor/evgeny1951mai' # начнем с того же дяденьки, но укажем, что нам нужно только 10 текстов: мы потестить
looklinks(start, viewedlinks, texts, 10)

https://proza.ru/avtor/evgeny1951mai
https://proza.ru/authors/
https://proza.ru/poems/
https://proza.ru/search.html
https://proza.ru/reg.html
https://proza.ru/texts/list.html
https://proza.ru/rating.html
https://proza.ru/2022/10/08/972
text for https://proza.ru/2022/10/08/972 added!
https://proza.ru/avtor/aleksandra34
https://proza.ru/recommend.html?aleksandra34
https://proza.ru/authors/password.html
https://proza.ru/rec_author.html?aleksandra34
https://proza.ru/avtor/ksark
https://proza.ru/recommend.html?ksark
https://proza.ru/rec_author.html?ksark
https://proza.ru/2022/09/24/1382
text for https://proza.ru/2022/09/24/1382 added!
https://proza.ru/readers.html?2022/09/24/1382
https://proza.ru/avtor/victorlev
https://proza.ru/recommend.html?victorlev
https://proza.ru/rec_author.html?victorlev
https://proza.ru/2020/10/03/928
text for https://proza.ru/2020/10/03/928 added!
https://proza.ru/readers.html?2020/10/03/928
https://proza.ru/avtor/dianakor
https://proza.ru/recommend.html?dianakor


In [ ]:
texts

Ну вот и все! осталось для красоты только переписать этот код на классах и в .py....... 